In [1]:
import torch
import torch.distributions as D

import SBM_SDE_tensor
import SBM_SDE
from obs_and_flow import *

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

dt = .1 #SDE discretization timestep.
t = 1 #Simulation run for T hours.
n = int(t / dt) + 1 
t_span = np.linspace(0, t, n)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
pretrain_lr = 1e-3
elbo_lr = 1e-3
niter = 1000
piter = 2
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
obs_error_scale = 0.1

In [3]:
temp_ref = 283
temp_rise = 5

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.05
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain temperature forcing function.
temp_tensor = SBM_SDE.temp_gen(t_span_tensor, temp_ref, temp_rise)

#Obtain SOC and DOC pool litter input vectors for use in flow SDE functions.
i_s_tensor = SBM_SDE.i_s(t_span_tensor) #Exogenous SOC input function
i_d_tensor = SBM_SDE.i_d(t_span_tensor) #Exogenous DOC input function

In [5]:
x_single = torch.zeros([1, 11, 3])
print(x_single.size())
x_batch = x_single.expand([2, 11, 3]).clone()
print(x_batch.size())

SOC = torch.normal(mean = torch.linspace(45, 46.1, 11), std = 1)
print(SOC)
DOC = torch.normal(mean = torch.linspace(0.07, 0.081, 11), std = 0.01)
print(DOC)
MBC = torch.normal(mean = torch.linspace(0.7, 0.81, 11), std = 0.1)
print(MBC)

torch.Size([1, 11, 3])
torch.Size([2, 11, 3])
tensor([46.5410, 44.8166, 43.0412, 45.8984, 44.3555, 44.1514, 46.0633, 46.6080,
        45.1607, 45.5867, 45.5034])
tensor([0.0718, 0.0625, 0.0832, 0.0626, 0.0756, 0.0698, 0.0803, 0.0688, 0.0637,
        0.0836, 0.0956])
tensor([0.7940, 0.7885, 0.7412, 0.8594, 0.6150, 0.6759, 0.7639, 0.7052, 0.8399,
        0.6677, 0.8292])


In [6]:
x_single[:, :, 0] = SOC
x_single[:, :, 1] = DOC
x_single[:, :, 2] = MBC
print(x_single)

x_batch[:, :, 0] = SOC
x_batch[:, :, 1] = DOC
x_batch[:, :, 2] = MBC
print(x_batch)

tensor([[[46.5410,  0.0718,  0.7940],
         [44.8166,  0.0625,  0.7885],
         [43.0412,  0.0832,  0.7412],
         [45.8984,  0.0626,  0.8594],
         [44.3555,  0.0756,  0.6150],
         [44.1514,  0.0698,  0.6759],
         [46.0633,  0.0803,  0.7639],
         [46.6080,  0.0688,  0.7052],
         [45.1607,  0.0637,  0.8399],
         [45.5867,  0.0836,  0.6677],
         [45.5034,  0.0956,  0.8292]]])
tensor([[[46.5410,  0.0718,  0.7940],
         [44.8166,  0.0625,  0.7885],
         [43.0412,  0.0832,  0.7412],
         [45.8984,  0.0626,  0.8594],
         [44.3555,  0.0756,  0.6150],
         [44.1514,  0.0698,  0.6759],
         [46.0633,  0.0803,  0.7639],
         [46.6080,  0.0688,  0.7052],
         [45.1607,  0.0637,  0.8399],
         [45.5867,  0.0836,  0.6677],
         [45.5034,  0.0956,  0.8292]],

        [[46.5410,  0.0718,  0.7940],
         [44.8166,  0.0625,  0.7885],
         [43.0412,  0.0832,  0.7412],
         [45.8984,  0.0626,  0.8594],
        

In [7]:
SCON_C_prior_means_tensor = {k: torch.tensor(v).expand(batch_size) for k, v in SCON_C_prior_means.items()}
print(SCON_C_prior_means)
print(SCON_C_prior_means_tensor)

{'u_M': 0.002, 'a_SD': 0.33, 'a_DS': 0.33, 'a_M': 0.33, 'a_MSC': 0.5, 'k_S_ref': 2.5e-05, 'k_D_ref': 0.005, 'k_M_ref': 0.0002, 'Ea_S': 75, 'Ea_D': 50, 'Ea_M': 50, 'c_SOC': 1.0, 'c_DOC': 0.01, 'c_MBC': 0.05}
{'u_M': tensor([0.0020, 0.0020]), 'a_SD': tensor([0.3300, 0.3300]), 'a_DS': tensor([0.3300, 0.3300]), 'a_M': tensor([0.3300, 0.3300]), 'a_MSC': tensor([0.5000, 0.5000]), 'k_S_ref': tensor([2.5000e-05, 2.5000e-05]), 'k_D_ref': tensor([0.0050, 0.0050]), 'k_M_ref': tensor([0.0002, 0.0002]), 'Ea_S': tensor([75, 75]), 'Ea_D': tensor([50, 50]), 'Ea_M': tensor([50, 50]), 'c_SOC': tensor([1., 1.]), 'c_DOC': tensor([0.0100, 0.0100]), 'c_MBC': tensor([0.0500, 0.0500])}


In [8]:
drift_single, diffusion_sqrt_single = SBM_SDE.drift_diffusion_SCON_C(x_single[:, :-1, :], t_span_tensor[:, :-1, :], i_s_tensor[:, :-1, :], i_d_tensor[:, :-1, :], temp_tensor[:, :-1, :], temp_ref, SCON_C_prior_means)
print(drift_single)
print(diffusion_sqrt_single)

drift_batch, diffusion_sqrt_batch = SBM_SDE_tensor.drift_diffusion_SCON_C(x_batch[:, :-1, :], t_span_tensor[:, :-1, :], i_s_tensor[:, :-1, :], i_d_tensor[:, :-1, :], temp_tensor[:, :-1, :], temp_ref, SCON_C_prior_means_tensor)
print(drift_batch)
print(diffusion_sqrt_batch)

tensor([[[-1.8820e-05,  7.4221e-06, -1.5155e-05],
         [-2.2201e-05,  6.3412e-05, -3.5767e-05],
         [ 2.6883e-05, -9.7014e-05,  1.2228e-05],
         [-1.1375e-04,  8.6632e-05, -5.7130e-05],
         [-9.0245e-05, -2.6662e-05,  1.8230e-05],
         [-1.2640e-04,  2.1692e-05, -9.3975e-06],
         [-1.9522e-04, -3.0849e-05, -1.1108e-05],
         [-2.7263e-04,  6.6772e-05, -2.4071e-05],
         [-2.6996e-04,  1.0327e-04, -6.8797e-05],
         [-2.8952e-04, -5.0080e-05,  8.2226e-06]]])
tensor([[[[1.0000, 0.0000, 0.0000],
          [0.0000, 0.1000, 0.0000],
          [0.0000, 0.0000, 0.2236]],

         [[1.0000, 0.0000, 0.0000],
          [0.0000, 0.1000, 0.0000],
          [0.0000, 0.0000, 0.2236]],

         [[1.0000, 0.0000, 0.0000],
          [0.0000, 0.1000, 0.0000],
          [0.0000, 0.0000, 0.2236]],

         [[1.0000, 0.0000, 0.0000],
          [0.0000, 0.1000, 0.0000],
          [0.0000, 0.0000, 0.2236]],

         [[1.0000, 0.0000, 0.0000],
          [0.0000, 0.1